In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# one document in the corpus is a book
from adhtools.utils import corpus_wordlist
from nlppln.utils import get_files

#xml_file1 = '/home/jvdzwaan/data/tmp/adh/chapters/1266MuhammadHasanNajafiJawhari.JawahirKalam.xml'
#xml_file2 = '/home/jvdzwaan/data/tmp/adh/chapters/0381IbnBabawayh.Hidaya.xml'

in_dir = '/home/jvdzwaan/Downloads/2019-02-08-fiqh-newfiles-alkhalil/'
        
in_files = get_files(in_dir)
c = corpus_wordlist(in_files, analyzer=True, field='word')

In [ ]:
%%time
from tqdm import tqdm_notebook as tqdm
data = [list(terms) for terms in tqdm(c, total=len(in_files))]

In [ ]:
corpus_file = 'fiqh-works-alkhalil.pkl'

In [ ]:
%%time
import pickle

with open(corpus_file, 'wb') as f:
    pickle.dump(data, f)

In [ ]:
%%time
import pickle

with open(corpus_file, 'rb') as f:
    data = pickle.load(f)

In [ ]:
#calculate pwc per book
df = calculate(data, w=0.001, top_k=200)

In [ ]:
df

In [ ]:
# selected works: 
#0483 Sarakhsī, Mabsūṭ
#0450 Māwardī, Ḥāwī
#0684 Qarāfi, Dhakhīra
#0884 Ibn Mufliḥ, Mubdiʿ
#0620 Ibn Qudāma, Mughnī
#0460 Ṭūsī, Mabsūṭ
#0676 Ḥillī, Sharāʾiʿ

import os

indices = {}
for i, in_file in enumerate(in_files):
    
    bn = os.path.basename(in_file)
    for num in ('0483', '0450', '0684', '0884', '0620', '0460', '0676'):
        if bn.startswith(num):
            print(i, os.path.splitext(bn)[0])
            indices[i] = os.path.splitext(bn)[0]

In [ ]:
indices

In [ ]:
del(indices[21])
indices

In [ ]:
# select columns
cols1 = ['{}_p'.format(k) for k in indices.keys()]
cols2 = ['{}_term'.format(k) for k in indices.keys()]

print(cols1)


cols = []
for c1, c2 in zip(cols1, cols2):
    cols.append(c1)
    cols.append(c2)
print(cols)    
df[cols]

In [ ]:
# rename columns

cols1 = ['{}_p'.format(v) for k, v in indices.items()]
cols2 = ['{}_term'.format(v) for k, v in indices.items()]

name_cols = []
for c1, c2 in zip(cols1, cols2):
    name_cols.append(c1)
    name_cols.append(c2)
print(name_cols) 

df = df[cols]
df.columns = name_cols
df

In [ ]:
df.to_csv('pwc-0.001-fiqh-works.csv')

In [ ]:
%%time
# one document in the corpus is a school
import os
import codecs
import glob

from itertools import chain

import pandas as pd

from nlppln.utils import get_files

from adhtools.utils import read_file_analyzer, read_file_stemmer

#xml_file1 = '/home/jvdzwaan/data/tmp/adh/chapters/1266MuhammadHasanNajafiJawhari.JawahirKalam.xml'
#xml_file2 = '/home/jvdzwaan/data/tmp/adh/chapters/0381IbnBabawayh.Hidaya.xml'

md_file = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/Meta/Metadata_Fiqh.csv'

in_dir = '/home/jvdzwaan/data/tmp/adh/2019-02-08-fiqh-newfiles-alkhalil-chapters/'

metadata = pd.read_csv(md_file, encoding='utf-8', sep=';|,')
#print(metadata.head())
schools = metadata.groupby('BookSUBJ')


def corpus(schools, in_dir, analyzer=True):

    for i, (name, data) in enumerate(schools):
        print(i, name)
        #print(data['BookURI'])
        words = []
        #with codecs.open('{}.txt'.format(i), 'w', encoding='utf-8') as f:
            
        for book in data['BookURI']:
            #print(book)
            #in_file = os.path.join(in_dir, '{}.xml'.format(book))
            in_files = glob.glob('{}/{}*.xml'.format(in_dir, book))
            #print(in_files)
            for in_file in in_files:
                if analyzer:
                    ws = read_file_analyzer(in_file)
                else:
                    ws = read_file_stemmer(in_file)
               #print(ws[0])
                words.append(ws)
        
        yield(chain(*words))
                #print(len(ws))
                #print(ws[0])
                #f.write(' '.join(ws))
                #f.write('\n')
            
c = corpus(schools, in_dir, analyzer=True)

In [ ]:
%%time
# make a sample for tuning the parameter. We use one book per school.
import os
import codecs
import glob

from itertools import chain
from collections import defaultdict

import pandas as pd

from nlppln.utils import get_files

from adhtools.utils import read_file_analyzer, read_file_stemmer

#xml_file1 = '/home/jvdzwaan/data/tmp/adh/chapters/1266MuhammadHasanNajafiJawhari.JawahirKalam.xml'
#xml_file2 = '/home/jvdzwaan/data/tmp/adh/chapters/0381IbnBabawayh.Hidaya.xml'

md_file = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/Meta/Metadata_Fiqh.csv'

in_dir = '/home/jvdzwaan/data/tmp/adh/2019-02-08-fiqh-newfiles-alkhalil-chapters/'

metadata = pd.read_csv(md_file, encoding='utf-8')
#print(metadata.head())
schools = metadata.groupby('BookSUBJ')

for i, (name, data) in enumerate(schools):
    # we use the oldest book for every school
    print(name)
    #print(data)
    for book in data.iterrows():
        # and sample 30.000 words
        b = book[1]
        #print(book[1])
        print(b['BookURI'], b['Number_of_tokens'])
        
        in_files = glob.glob('{}/{}*.xml'.format(in_dir, book))
        #print(in_files)
        counts = defaultdict(int)
        for in_file in in_files:
            if analyzer:
                ws = read_file_analyzer(in_file)
            else:
                ws = read_file_stemmer(in_file)
            for word in ws:
                counts[word] += 1
                
                    
        
        break


In [ ]:
%%time
# make a sample for tuning the parameter. We use one book per school.
import os
import codecs
import glob

from itertools import chain

import pandas as pd

from nlppln.utils import get_files

from adhtools.utils import read_file_analyzer, read_file_stemmer

#xml_file1 = '/home/jvdzwaan/data/tmp/adh/chapters/1266MuhammadHasanNajafiJawhari.JawahirKalam.xml'
#xml_file2 = '/home/jvdzwaan/data/tmp/adh/chapters/0381IbnBabawayh.Hidaya.xml'

md_file = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/Meta/Metadata_Fiqh.csv'

in_dir = '/home/jvdzwaan/data/tmp/adh/2019-02-08-fiqh-newfiles-alkhalil-chapters/'

metadata = pd.read_csv(md_file, encoding='utf-8')
#print(metadata.head())
schools = metadata.groupby('BookSUBJ')


def corpus(schools, in_dir, analyzer=True):

    for i, (name, data) in enumerate(schools):
        print(i, name)
        #print(data['BookURI'])
        words = []
        #with codecs.open('{}.txt'.format(i), 'w', encoding='utf-8') as f:
            
        for book in data['BookURI']:
            #print(book)
            #in_file = os.path.join(in_dir, '{}.xml'.format(book))
            in_files = glob.glob('{}/{}*.xml'.format(in_dir, book))
            #print(in_files)
            for in_file in in_files:
                if analyzer:
                    ws = read_file_analyzer(in_file)
                else:
                    ws = read_file_stemmer(in_file)
               #print(ws[0])
                words.append(ws)
        
        yield(chain(*words))
                #print(len(ws))
                #print(ws[0])
                #f.write(' '.join(ws))
                #f.write('\n')
            
c = corpus(schools, in_dir, analyzer=True)

In [ ]:
words = read_file('/home/jvdzwaan/data/tmp/adh/analysis/alkhalil/0311AbuBakrKhallal.WuqufWaTarajjul.xml')
print(len(words))

In [ ]:
def read_text_file(in_file):
    with codecs.open(in_file, encoding='utf-8') as f:
        for ln in f:
            for word in ln.split():
                #print(word)
                yield word

In [ ]:
%%time
data = [list(terms) for terms in c]

In [ ]:
corpus_file = 'fiqh-schools-alkhalil.pkl'

In [ ]:
%%time
import pickle

with open(corpus_file, 'wb') as f:
    pickle.dump(data, f)

In [ ]:
%%time
import pickle

with open(corpus_file, 'rb') as f:
    data = pickle.load(f)

In [ ]:
for terms in data:
    print(len(terms))

In [ ]:
print(data[0][0])

In [ ]:
%%time
from weighwords import ParsimoniousLM

model = ParsimoniousLM([terms for terms in data], w=.01)

In [ ]:
%%time

dfs = []

top_k = 25
for i, terms in enumerate(data):
    result = []
    for term, p in model.top(top_k, terms, max_iter=100):
        result.append({'{}_term'.format(i): term, '{}_p'.format(i): np.exp(p)})
        #print("    %s %.4f" % (term, np.exp(p)))
        #print(term)
    dfs.append(pd.DataFrame(result))
    #print('---')

In [ ]:
pd.concat(dfs, axis=1, sort=False)

In [ ]:
from weighwords import ParsimoniousLM

def calculate(data, w, top_k=25):
    model = ParsimoniousLM([terms for terms in data], w=w)
    
    # calculate terms and weights
    dfs = []

    for i, terms in enumerate(data):
        result = []
        for term, p in model.top(top_k, terms, max_iter=100):
            result.append({'{}_term'.format(i): term, '{}_p'.format(i): np.exp(p)})
        dfs.append(pd.DataFrame(result))
    return pd.concat(dfs, axis=1, sort=False)

In [ ]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
%%time
wordcloud_data = []

weights = (1.0, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001)

for w in weights:
    wordcloud_data.append(calculate(data, w=w))

In [ ]:
%%time
# pwc6 (selected by Christian) for schools

df = calculate(data, w=0.001, top_k=200)

In [ ]:
label_0 = 'حنبلي'
label_1 = 'حنفي'
label_2 = 'شافعي'
label_3 = 'شيعي'
label_4 = 'مالكي'

labels = [label_0, label_1, label_2, label_3, label_4]

In [ ]:
df.columns = ['{}_p'.format(label_0), '{}_term'.format(label_0),
              '{}_p'.format(label_1), '{}_term'.format(label_1),
              '{}_p'.format(label_2), '{}_term'.format(label_2),
              '{}_p'.format(label_3), '{}_term'.format(label_3),
              '{}_p'.format(label_4), '{}_term'.format(label_4)]

In [ ]:
df

In [ ]:
df.to_csv('pwc-0.001-fiqh-schools.csv')

In [ ]:
def get_terms(txt_file):
    # get the terms list
    terms = pd.read_csv(txt_file, encoding='utf-8', index_col=None, header=None)
    t = terms[0].tolist()
    print('total number of terms:', len(t))
    terms = set(t)
    print('number of unique terms:', len(terms))
    return terms
stopwords = get_terms('/home/jvdzwaan/data/adh/stopwords/custom.txt')

In [ ]:
def sw(term):
    return 'background-color: yellow' if term in stopwords else ''

In [ ]:
print(len(wordcloud_data))

In [ ]:
wordcloud_data[0].style.applymap(sw)

In [ ]:
wordcloud_data[1].style.applymap(sw)

In [ ]:
wordcloud_data[2].style.applymap(sw)

In [ ]:
wordcloud_data[3].style.applymap(sw)

In [ ]:
wordcloud_data[4].style.applymap(sw)

In [ ]:
wordcloud_data[5].style.applymap(sw)

In [ ]:
wordcloud_data[6].style.applymap(sw)

In [ ]:
wordcloud_data[7].style.applymap(sw)

In [ ]:
wordcloud_data[8].style.applymap(sw)

In [ ]:
for i, w in enumerate(weights):
    wordcloud_data[i].style.applymap(sw).to_excel('pwc{}.xls'.format(i), engine='openpyxl')

In [ ]:
print('\n'.join(list(wordcloud_data[5]['0_term'])))

In [ ]:
%%time
import codecs

c_from_text = [read_text_file(t) for t in ('0.txt', '1.txt', '2.txt', '3.txt', '4.txt')]
[len(list(terms)) for terms in c_from_text]

In [ ]:
import os

top_k = 20
words = {}

for fname, doc in zip(in_files, terms):
    print("Top %d words in %s:" % (top_k, os.path.basename(fname)))
    words[os.path.basename(fname)] = {}
    for term, p in model.top(top_k, doc):
        print("    %s %.4f" % (term, np.exp(p)))
        words[os.path.basename(fname)][term] = np.exp(p)
print("")

In [ ]:
# Boek dat Christian belangrijk vindt
# 0620IbnQudamaMaqdisi.MughniFiFiqh.xml

In [ ]:
print(len(terms))

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(background_color="white", font_path='/usr/share/fonts/opentype/fonts-hosny-amiri/amiri-quran.ttf')
# generate word cloud
wc.generate_from_frequencies(words['0179MalikIbnAnas.Muwatta.xml'])

# show
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wc.generate_from_frequencies(words['0483IbnAhmadSarakhsi.Mabsut.xml'])

# show
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
for word, w in words['0483IbnAhmadSarakhsi.Mabsut.xml'].items():
    print(word, w)